# DataFrames

## Acquiring a DataFrame

### Convert Pandas DataFrame to Spark DataFrame

In [1]:
import pandas as pd
from random import randint

n_cols = 10
n_rows = 10

pdf = pd.DataFrame([tuple([c for c in range(n_cols)]) for r in range(n_rows)], columns=[f'x{i}' for i in range(n_cols)])
sdf = sqlContext.createDataFrame(pdf)

In [2]:
pdf

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0,1,2,3,4,5,6,7,8,9
1,0,1,2,3,4,5,6,7,8,9
2,0,1,2,3,4,5,6,7,8,9
3,0,1,2,3,4,5,6,7,8,9
4,0,1,2,3,4,5,6,7,8,9
5,0,1,2,3,4,5,6,7,8,9
6,0,1,2,3,4,5,6,7,8,9
7,0,1,2,3,4,5,6,7,8,9
8,0,1,2,3,4,5,6,7,8,9
9,0,1,2,3,4,5,6,7,8,9


In [3]:
sdf.collect()

[Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9)]

In [4]:
sdf.show()

+---+---+---+---+---+---+---+---+---+---+
| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+---+---+---+---+---+---+---+---+---+---+
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
+---+---+---+---+---+---+---+---+---+---+



In [5]:
sdf.printSchema()

root
 |-- x0: long (nullable = true)
 |-- x1: long (nullable = true)
 |-- x2: long (nullable = true)
 |-- x3: long (nullable = true)
 |-- x4: long (nullable = true)
 |-- x5: long (nullable = true)
 |-- x6: long (nullable = true)
 |-- x7: long (nullable = true)
 |-- x8: long (nullable = true)
 |-- x9: long (nullable = true)



### Convert a RDD to DataFrame

In [6]:
from random import randint
from pyspark.sql.types import *

n_cols = 10
n_rows = 10

rdd = sc.parallelize([[c for c in range(n_cols)] for r in range(n_rows)])

schema = StructType([StructField(f'x{i}', IntegerType(), True) for i in range(n_cols)])
df = sqlContext.createDataFrame(rdd, schema)

In [7]:
rdd.collect()

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [8]:
df.collect()

[Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9)]

In [9]:
df.show()

+---+---+---+---+---+---+---+---+---+---+
| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+---+---+---+---+---+---+---+---+---+---+
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
+---+---+---+---+---+---+---+---+---+---+



In [10]:
df.printSchema()

root
 |-- x0: integer (nullable = true)
 |-- x1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- x3: integer (nullable = true)
 |-- x4: integer (nullable = true)
 |-- x5: integer (nullable = true)
 |-- x6: integer (nullable = true)
 |-- x7: integer (nullable = true)
 |-- x8: integer (nullable = true)
 |-- x9: integer (nullable = true)



### Convert JSON data to Spark DataFrame

In [11]:
sql

<bound method SparkSession.sql of <pyspark.sql.session.SparkSession object at 0x7fb58827fa50>>

## DataFrame operations

### Create data

In [12]:
import pandas as pd
from random import randint

n_cols = 10
n_rows = 10

pdf = pd.DataFrame([tuple([randint(1, 10) for _ in range(n_cols)]) for r in range(n_rows)], columns=[f'x{i}' for i in range(n_cols)])
sdf = sqlContext.createDataFrame(pdf)

In [13]:
pdf

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,6,9,8,4,4,4,5,4,6,8
1,2,5,7,3,3,5,5,1,2,7
2,1,6,9,9,1,4,9,6,5,6
3,9,4,2,9,1,4,7,5,9,3
4,6,5,8,3,1,5,6,9,10,10
5,10,7,2,9,6,3,7,3,3,3
6,4,7,10,9,3,1,10,1,2,8
7,9,10,1,5,6,2,7,6,2,2
8,5,7,2,8,9,10,7,1,5,7
9,9,8,5,2,1,3,6,2,1,4


### Select

In [14]:
sdf.select('x0').show()

+---+
| x0|
+---+
|  6|
|  2|
|  1|
|  9|
|  6|
| 10|
|  4|
|  9|
|  5|
|  9|
+---+



### Select multiple columns

In [15]:
sdf.select(sdf['x0'], sdf['x1']).show()

+---+---+
| x0| x1|
+---+---+
|  6|  9|
|  2|  5|
|  1|  6|
|  9|  4|
|  6|  5|
| 10|  7|
|  4|  7|
|  9| 10|
|  5|  7|
|  9|  8|
+---+---+



### Select multiple columns and modify values

In [16]:
sdf.select(sdf['x0'] * 2, sdf['x1'] * 3).show()

+--------+--------+
|(x0 * 2)|(x1 * 3)|
+--------+--------+
|      12|      27|
|       4|      15|
|       2|      18|
|      18|      12|
|      12|      15|
|      20|      21|
|       8|      21|
|      18|      30|
|      10|      21|
|      18|      24|
+--------+--------+



### Filtering

In [17]:
sdf.select(sdf['x0'] > 5).show()

+---+---+---+---+---+---+---+---+---+---+
| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+---+---+---+---+---+---+---+---+---+---+
|  6|  9|  8|  4|  4|  4|  5|  4|  6|  8|
|  9|  4|  2|  9|  1|  4|  7|  5|  9|  3|
|  6|  5|  8|  3|  1|  5|  6|  9| 10| 10|
| 10|  7|  2|  9|  6|  3|  7|  3|  3|  3|
|  9| 10|  1|  5|  6|  2|  7|  6|  2|  2|
|  9|  8|  5|  2|  1|  3|  6|  2|  1|  4|
+---+---+---+---+---+---+---+---+---+---+



### Group by

In [18]:
sdf.groupBy('x0').count().show()

+---+-----+
| x0|count|
+---+-----+
|  6|    2|
|  9|    3|
|  5|    1|
|  1|    1|
| 10|    1|
|  2|    1|
|  4|    1|
+---+-----+

